<a href="https://colab.research.google.com/github/sheldon-o/movies-shows-recomendation/blob/master/api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#this project is using different websites api to access their data and recommend
#movies and web shows

#first we will use tastedrive api to fetch movies

#  https://tastedive.com/read/api     taste drive api
#  https://www.omdbapi.com/           OMDB api
#check these pages for keys of api's







 


In [ ]:
pip install requests

In [36]:
import requests
import json


In [94]:
def get_movies_from_taste_drive(str):
  PARAMS = {}

  PARAMS['q']=str
  PARAMS['type'] = "movies"
  PARAMS['limit'] = 5
  #PARAMS = {'q':q,'type':type,'limit':limit}
  key = "https://tastedive.com/api/similar"
  resp = requests.get(key, params=PARAMS)
  print(resp)
  print(resp.url)

  
 # print("test")
  return resp.json()
  


In [96]:
get_movies_from_taste_drive("Black Panther")

<Response [200]>
https://tastedive.com/api/similar?q=Black+Panther&type=movies&limit=5


{'Similar': {'Info': [{'Name': 'Black Panther', 'Type': 'movie'}],
  'Results': [{'Name': 'Guardians Of The Galaxy Vol. 2', 'Type': 'movie'},
   {'Name': 'Thor: Ragnarok', 'Type': 'movie'},
   {'Name': 'Ant-Man And The Wasp', 'Type': 'movie'},
   {'Name': 'Rampage', 'Type': 'movie'},
   {'Name': 'Deadpool 2', 'Type': 'movie'}]}}

now from the output we will fetch the movies name only : 


In [61]:
def extract_movies_titles(func):
  nest1 = func['Similar']
  nest2 = nest1['Results']
  output=[]
  for i in nest2:
    output.append(i['Name'])

 # print(output)
  return output

 
extract_movies_titles(get_movies_from_taste_drive("Black Panther"))
extract_movies_titles(get_movies_from_taste_drive("Captain Marvel"))


<Response [200]>
https://tastedive.com/api/similar?q=Black+Panther&type=movies&limit=5
test
['Guardians Of The Galaxy Vol. 2', 'Thor: Ragnarok', 'Ant-Man And The Wasp', 'Rampage', 'Deadpool 2']
<Response [200]>
https://tastedive.com/api/similar?q=Captain+Marvel&type=movies&limit=5
test
['Avengers: Infinity War', 'Aquaman', 'Avengers: Endgame', 'Spider-Man: Far From Home', 'Ant-Man And The Wasp']


['Avengers: Infinity War',
 'Aquaman',
 'Avengers: Endgame',
 'Spider-Man: Far From Home',
 'Ant-Man And The Wasp']

for some movies we get a recommendation more than once so we have to fix it


In [107]:
def get_related_titles(lst):
  output=[]
  for i in lst:
    gmftd = get_movies_from_taste_drive(i)
    emt = extract_movies_titles(gmftd)
    for j in emt:
      if j not in output:
        output.append(j)
  return output
get_related_titles(["Bridesmaids", "Sherlock Holmes"])


<Response [200]>
https://tastedive.com/api/similar?q=Bridesmaids&type=movies&limit=5
['Bachelorette', 'Baby Mama', 'Wanderlust', 'The Five-Year Engagement', 'Bride Wars']
<Response [200]>
https://tastedive.com/api/similar?q=Sherlock+Holmes&type=movies&limit=5
['Captain America', 'Sherlock Holmes: A Game Of Shadows', 'Batman And Robin', 'Men In Black', 'Up!']


['Bachelorette',
 'Baby Mama',
 'Wanderlust',
 'The Five-Year Engagement',
 'Bride Wars',
 'Captain America',
 'Sherlock Holmes: A Game Of Shadows',
 'Batman And Robin',
 'Men In Black',
 'Up!']

In [85]:
#this function will take a string (moviename) and return information about the
# movie in dictionary format
def get_movie_data(str):
  #baseurl = "http://www.omdbapi.com/"
  baseurl ="http://www.omdbapi.com?apikey=eacb9159"
  PARAMS = {}
  PARAMS['t']=str
  PARAMS['r']="json"
  resp = requests.get(baseurl, params=PARAMS)
  return resp.json()
print(get_movie_data("Venom"))



{'Title': 'Venom', 'Year': '2018', 'Rated': 'PG-13', 'Released': '05 Oct 2018', 'Runtime': '112 min', 'Genre': 'Action, Adventure, Sci-Fi', 'Director': 'Ruben Fleischer', 'Writer': "Jeff Pinkner (screenplay by), Scott Rosenberg (screenplay by), Kelly Marcel (screenplay by), Jeff Pinkner (screen story by), Scott Rosenberg (screen story by), Todd McFarlane (Marvel's Venom Character created by), David Michelinie (Marvel's Venom Character created by)", 'Actors': 'Tom Hardy, Michelle Williams, Riz Ahmed, Scott Haze', 'Plot': 'A failed reporter is bonded to an alien entity, one of many symbiotes who have invaded Earth. But the being takes a liking to Earth and decides to protect it.', 'Language': 'English, Mandarin, Malay', 'Country': 'China, USA', 'Awards': '3 wins & 9 nominations.', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNzAwNzUzNjY4MV5BMl5BanBnXkFtZTgwMTQ5MzM0NjM@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.7/10'}, {'Source': 'Rotten Tomatoes'

In [105]:
#this function will take output of get_movie_data and
#return ratings of movie

def get_movie_rating(dic):
  print(dic.keys())
  nest = dic["Ratings"]
  print(nest)
  for i in nest:
    if i['Source']=="Rotten Tomatoes":

      return int(i["Value"].strip("%"))
  return 0

  
#get_movie_rating(get_movie_data("venom"))


It takes a list of movie titles as an input. It returns a sorted list of related movie titles as output, up to five related movies for each input movie title. The movies should be sorted in descending order by their Rotten Tomatoes rating, as returned by the get_movie_rating


In [113]:


def get_sorted_recommendations(lst):
  final={}
  nest = get_related_titles(lst)
  for i in nest:
    movie_ratings = get_movie_rating(get_movie_data(i))
    final[i]=movie_ratings
  output = sorted(final.items(),key = lambda item:(item[1],item[0]), reverse = True)
  result = []
  for i in output:
    result.append(i[0])
  print(result)

get_sorted_recommendations(["Bridesmaids", "Sherlock Holmes"])




<Response [200]>
https://tastedive.com/api/similar?q=Bridesmaids&type=movies&limit=5
['Bachelorette', 'Baby Mama', 'Wanderlust', 'The Five-Year Engagement', 'Bride Wars']
<Response [200]>
https://tastedive.com/api/similar?q=Sherlock+Holmes&type=movies&limit=5
['Captain America', 'Sherlock Holmes: A Game Of Shadows', 'Batman And Robin', 'Men In Black', 'Up!']
dict_keys(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster', 'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type', 'DVD', 'BoxOffice', 'Production', 'Website', 'Response'])
[{'Source': 'Internet Movie Database', 'Value': '5.3/10'}, {'Source': 'Rotten Tomatoes', 'Value': '56%'}, {'Source': 'Metacritic', 'Value': '53/100'}]
dict_keys(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster', 'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbI